In [2]:
import fitz
import pandas as pd
import re

## Hoffmann-Kunze

In [3]:
doc = fitz.open("../dat/books/all_books/textbook_Hoffman_Kunze.pdf")

In [6]:
concepts = []
last_x = parent = is_indented = inverted = False
last_concept = ""

for page_num in range(409,416):
    print("Reached page " + str(page_num))
    
    for word in doc[page_num].get_text("blocks", sort="blocks"):
        if re.search(r"^\w \n$", word[4]): continue
        content = word[4].strip()
        if content == "Index": continue
        if content.find("<image") != -1: continue

        current_x = round(word[0] / 4) * 4
        delta_x = current_x - last_x
        last_x = current_x

        if last_x and delta_x > 5 and delta_x < 15:
            is_indented = True
            inverted = (last_concept.find(":") == -1)
            parent = last_concept.replace(":", "").strip()
        if last_x and (delta_x < -5 or delta_x > 15):
            is_indented = False
            parent = False

        info = content.split(", ")
        start = 1
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        
        concept = ", ".join(info[0:start]).replace(" (cont,)","").replace(" (cont.)","")
        last_concept = concept.strip()
        pages = info[start:]

        pages = [i.replace(" ","") for i in pages]
        regex = re.compile(r"^\d+")
        
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        if None in pages: continue
        
        if len(info) == 1: continue
        starts_with_of = concept[0:2] == "of"
        ends_with_of = concept[-2:] == "of"
        composed = (concept + " " + parent if (inverted or ends_with_of) and not starts_with_of else parent+" "+concept) if is_indented else concept 
        if parent and concept.find(parent) != -1: composed = concept
        
        print(composed + " on " + " and ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': 2 if parent else 1})


Reached page 409
Adjoint classical on 148 and 159
Adjoint of transformation on 295
Admissible subspace on 232
Algebra on 117
Algebra of formal power serie� on 119
self-adjoint Algebra on 345
Algebraically closed field on 138
Alternating n-linear function on 144 and 169
Annihilator of subset on 101
Annihilator of sum and intersection on 106
Annihilator of vector ( T-annihilator) on 201 and 202 and 228
Approximation on 283
Associativity on 1
Associativity of matrix multiplication on 19 and 90
Associativity of vector addition on 28
Augmented matrix on 14
Axiom o f  choice on 400
Basis on 41
change of Basis on 92
dual Basis on 99 and 165
for module Basis on 164
ordered Basis on 50
orthonormal Basis on 281
standard b!4�is of l!'n Basis on 41
Bessel's inequality on 287
Bilinear form on 166 and 320 and 359
diagonalization of Bilinear form on 370
group preserving Bilinear form on 379
matrix of Bilinear form on 362
non-degenerate ( non-singular) Bilinear form on 365
positive definite Bilinear f

In [7]:
index_data = pd.DataFrame(concepts)
index_data

,concept,pages,level
0,Adjoint classical,"148, 159",2
1,Adjoint of transformation,295,2
2,Admissible subspace,232,1
3,Algebra,117,1
4,Algebra of formal power serie�,119,2
...,...,...,...
470,quotient of Vector space,397,2
471,of solutions to linear equations,36,1
472,Wedge (exterior) product,"175, 177",1
473,Zero matrix,12,2


In [8]:
index_data.to_csv("../dat/index/textbook_Hoffman_Kunze.csv")

## Nicholson

In [9]:
doc = fitz.open("../dat/books/all_books/Nicholson-OpenLAWA-2019A.pdf")

In [10]:
concepts = []
line_x = 0
last_x = parent = is_indented = inverted = False
last_concept = buffer = content = ""

for page_num in range(686,697):
    print("Reached page " + str(page_num))
    
    words = [word for word in doc[page_num].get_text("words", sort="blocks")]
    for idx, word in enumerate(words):
        if buffer == "": line_x = word[0]
        current_y = round(word[1]/6)
        next_y = round(words[idx+1][1]/6) if idx+1 < len(words) else 0
        buffer += word[4] + " "
        if current_y != next_y:
            content = buffer
            buffer = ""
        else:
            continue
        
        if content[-2:] == ", ":
            buffer = content
            continue

        if re.search(r"\D$", content.strip()) is not None and idx+1 < len(words):
            if round(word[0]/6) - round(words[idx+1][0]/6) > 4:
                buffer = content.replace("- ", "")
                continue
        #print(content)

        if content == "Index": continue
        #if content.find("<image") != -1: continue

        current_x = round(line_x / 6)
        delta_x = current_x - last_x
        last_x = current_x
        #print(delta_x)
        if last_x and delta_x > 1 and delta_x < 3:
            is_indented = True
            inverted = False
            parent = last_concept.replace(":", "").strip()
        if last_x and (delta_x < -1 or delta_x > 3):
            is_indented = False
            parent = False

        info = content.split(", ")
        start = 1
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        
        concept = ", ".join(info[0:start]).replace(" (cont,)","").replace(" (cont.)","")
        last_concept = concept.strip()
        pages = info[start:]

        pages = [i.replace(" ","") for i in pages]
        regex = re.compile(r"^\d+")
        
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        if None in pages: continue
        
        if len(info) == 1: continue
        starts_with_of = concept[0:2] == "of"
        ends_with_of = concept[-2:] == "of"
        composed = (concept + " " + parent if (inverted or ends_with_of) and not starts_with_of else parent+", "+concept) if is_indented else concept 
        if parent and concept.find(parent) != -1: composed = concept
        
        print(composed + " on " + " and ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': 2 if parent else 1})


Reached page 686
(i, j)-entry on 35
 on 3 and 58
A-invariance on 176
B-matrix on 513
T-invariant on 523
m× n matrix, canonical forms on 591
m× n matrix, deﬁned on 35
m× n matrix, difference on 38
m× n matrix, elementary row operation on 96
m× n matrix, main diagonal on 43
m× n matrix, matrix transformation on 503
m× n matrix, negative on 38
m× n matrix, subspaces on 265
m× n matrix, transpose on 42
m× n matrix, zero matrix on 38
n-parity-check code on 479
n-tuples on 263 and 290 and 330
n-vectors on 47
n-words on 477
nth roots of unity on 605
r-ball on 478
x-axis on 209
x-compression on 61
x-expansion on 61
x-shear on 61
y-axis on 209
y-compression on 61
y-expansion on 61
z-axis on 209
Disquisitiones Arithmeticae (Gauss) on 11
How to Read and Do Proofs (Solow) on 611
Introduction to Abstract Algebra (Nicholson) on 475
Introduction to Abstract Mathematics (Lucas) on 611
Introduction to the Theory of Error-Correcting Codes (Pless) on 484
Mécanique Analytique (Lagrange) on 245
Calcolo Geo

In [11]:
index_data = pd.DataFrame(concepts)
index_data

,concept,pages,level
0,"(i, j)-entry",35,1
1,,"3, 58",1
2,A-invariance,176,1
3,B-matrix,513,1
4,T-invariant,523,1
...,...,...,...
1663,zero rows,10,1
1664,zero subspace,"263, 339",1
1665,zero transformation,"60, 117, 376",1
1666,zero vector,"263, 330",1


In [12]:
index_data.to_csv("../dat/index/Nicholson-OpenLAWA-2019A.csv")

## Kuttler

In [14]:
doc = fitz.open("../dat/books/all_books/Kuttler-LinearAlgebra-AFirstCourse-2017A.pdf")

In [32]:
concepts = []
line_x = 0
last_x = parent = False
last_concept = buffer = content = ""

for page_num in range(602,606):
    print("Reached page " + str(page_num))
    
    words = [word for word in doc[page_num].get_text("words", sort="blocks")] # to be able to look ahead
    for idx, word in enumerate(words):
        if buffer == "": line_x = word[0]
        
        if word[4].find("INDEX") != -1: 
            buffer = ""
            continue
        
        buffer += word[4] + " "

        current_y = round(word[1]/6)
        next_y = round(words[idx+1][1]/6) if idx+1 < len(words) else 0
        if current_y != next_y:
            content = buffer
            buffer = ""
        else: continue
        #print(content)

        current_x = round(line_x / 6)
        delta_x = current_x - last_x
        last_x = current_x
        #print(delta_x)

        if delta_x > 1 and delta_x < 5:
            parent = last_concept.strip() 
        if delta_x < -1 and delta_x > -5:
            parent = False

        info = content.split(", ")
        start = 1
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        
        concept = ", ".join(info[0:start]).strip()
        last_concept = concept

        pages = [i.replace(" ","") for i in info[start:]]
        regex = re.compile(r"^\d+")
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        
        if len(info) == 1: continue

        composed = parent + ", " + concept if parent else concept
        
        print(composed + ": " + ", ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': 2 if parent else 1})


Reached page 602
∩: 537
∪: 537
\: 537
row-echelon form: 16
reduced row-echelon form: 16
reduced row-echelon form, algorithm: 19
adjugate: 130
back substitution: 13
base case: 540
basic solution: 29
basis: 203, 204, 234, 487
basis, any two same size: 489
box product: 188
cardioid: 443
Cauchy Schwarz inequality: 168
characteristic equation: 349
chemical reactions, balancing: 33
Cholesky factorization, positive deﬁnite: 420
classical adjoint: 130
Cofactor Expansion: 110
cofactor matrix: 130
column space: 211
complex eigenvalues: 369
complex numbers, absolute value: 334
complex numbers, addition: 329
complex numbers, argument: 336
complex numbers, conjugate: 331
complex numbers, conjugate of a product: 336
complex numbers, modulus: 334, 336
complex numbers, multiplication: 330
complex numbers, polar form: 336
complex numbers, roots: 339
complex numbers, standard form: 329
complex numbers, triangle inequality: 334
component form: 162
component of a force: 265, 267
consistent system: 8
coord

In [33]:
index_data = pd.DataFrame(concepts)
print(index_data)
index_data.to_csv("../dat/index/Kuttler-LinearAlgebra-AFirstCourse-2017A.csv")

                      concept     pages  level
0                           ∩       537      1
1                           ∪       537      1
2                           \       537      1
3            row-echelon form        16      1
4    reduced row-echelon form        16      1
..                        ...       ...    ...
297                      work       265      1
298               zero matrix        54      1
299             zero subspace       201      1
300       zero transformation  271, 500      1
301               zero vector       149      1

[302 rows x 3 columns]


## Math 1410

In [34]:
doc = fitz.open("../dat/books/all_books/Math1410_print.pdf")

In [63]:
concepts = []
line_x = indent_lvl = 0
last_x = False
parents = []
last_concept = buffer = content = ""

for page_num in range(402,406):
    print("Reached page " + str(page_num))
    
    words = [word for word in doc[page_num].get_text("words", sort="blocks")] # to be able to look ahead
    for idx, word in enumerate(words):
        if buffer == "": line_x = word[0]
        
        if word[4].find("INDEX") != -1 or word[4].find("A.��") != -1: 
            #print(word[4])
            buffer = ""
            continue
        
        buffer += word[4] + " "

        current_y = round(word[1]/6)
        next_y = round(words[idx+1][1]/6) if idx+1 < len(words) else 0
        if current_y != next_y:
            content = buffer
            buffer = ""
        else: continue
        #print(content)

        current_x = round(line_x / 6)
        delta_x = current_x - last_x
        last_x = current_x
        #print(delta_x)

        if delta_x > 1 and delta_x < 5 or last_concept == "distance":
            indent_lvl += 1
            #print("Indent level: " + str(indent_lvl))
            parents.append(last_concept)
            
        if delta_x < -1 and delta_x > -5 or last_concept == "standard matrix of":
            indent_lvl -= 1
            #print("Indent level: " + str(indent_lvl))
            parents = parents[0:indent_lvl]

        if delta_x < -4 and delta_x > -8:
            indent_lvl -= 2
            #print("Indent level: " + str(indent_lvl))
            parents = parents[0:indent_lvl]

        info = content.split(", ")
        start = False
        for idx, i in enumerate(info):
            if re.search(r"^\D", i) is None:
                start = idx
                break
        if not start:
            concept = ", ".join(info).strip()
            last_concept = concept
            continue
        
        concept = ", ".join(info[0:start]).strip()
        last_concept = concept

        pages = [i.replace(" ","") for i in info[start:]]
        regex = re.compile(r"^\d+")
        pages = [regex.findall(i)[0] if len(regex.findall(i)) else None for i in pages]
        
        if len(info) == 1: continue

        composed = ""
        if indent_lvl == 2: 
            composed += parents[0] + ", " + parents[1] + ", "
        if indent_lvl == 1:
            composed += parents[0] + ", "
        composed += concept
        
        print(composed + ": " + ", ".join(pages))
        concepts.append({'concept': composed, 'pages': ", ".join(pages), 'level': indent_lvl+1})


Reached page 402
∈: 2
/∈: 2
nth Roots of Unity: 49
nth root, of a complex number: 44, 45
x-axis: 12
x-coordinate: 12
y-axis: 12
y-coordinate: 12
abscissa: 12
absolute value: 14
adjugate: 337
antisymmetric: 305
argument, of a complex number, definition of: 35
argument, of a complex number, properties of: 38
augmented matrix: 129
back substitution: 138
basic solution: 177
basic variable: 150
basis: 282
basis, of a null space: 291
Cartesian coordinate plane: 12
Cartesian coordinates: 12
characteristic polynomial: 346, 369
cis(θ): 39
cofactor: 314
cofactor, expansion: 316, 322
cofactor, matrix: 337
column, pivot: 288
column space: 287
column vector: 191
Complex Factorization Theorem: 26
complex number, nth root: 44, 45
complex number, nth Roots of Unity: 49
complex number, argument, definition of: 35
complex number, argument, properties of: 38
complex number, complex conjugate, definition of: 21
complex number, conjugate, properties of: 22
complex number, definition of: 19, 35
complex numb

In [64]:
index_data = pd.DataFrame(concepts)
print(index_data)
index_data.to_csv("../dat/index/Math1410_print.csv")

                           concept   pages  level
0                                ∈       2      1
1                               /∈       2      1
2               nth Roots of Unity      49      1
3    nth root, of a complex number  44, 45      2
4                           x-axis      12      1
..                             ...     ...    ...
297           vectors, unit vector  60, 62      2
298           vectors, zero vector      58      2
299                   Venn Diagram       3      1
300                           work      75      1
301                    zero matrix     196      1

[302 rows x 3 columns]
